### Trajectory Optimization
1. Trajectory optimization is concerned with finding the optimal trajectory given some objective functions, for example minimal force or minimal work:
<br> $$\min_{u(t),x(t),v(t)} \int_{0}^1 u^2 (\tau) d\tau$$
<br> $$\min_{u(t),x(t),v(t)} \int_{0}^1 |u (\tau) v(\tau)| d\tau$$

2. A standard trajectory optimization problem consists of two terms, a *mayer* term that captures boundary conditions and a *lagrange* term that is a path integral along the entire trajectory:
<br> $$\min_{t_0,t_f,u(t),\mathbf{x}(t),\mathbf{v}(t)} J(t_0,t_f,\mathbf{x}(t_0),\mathbf{x}(t_f)) + \int_{t_0}^{t_f} w(\tau,\mathbf{x}(\tau),\mathbf{u}(\tau)) d\tau$$

#### Direct Collocation Methods
1. Direct collocation operates by transcribing a trajectory generation problem in to a nonlinear program.

2. Simple example: Consider moving a unit mass from rest at $x=0$ to stop at $x=1$ in $1$ second.
<br> $$\min_{u(t),x(t),v(t)} \int_{0}^1 u^2 (\tau) d\tau$$
<br> $$u^* = 6-12t, x^* = 3t^2 - 2t^3$$
where the second euqation is found from calculus of variations

3. To begin, we need to discretize the trajectory, which gives a finite set of decision variables. This is done by setting up collocation points:
$$t \rightarrow t_0 ... t_k ... t_N$$
$$x \rightarrow x_0 ... x_k ... x_N$$
$$v \rightarrow v_0 ... v_k ... v_N$$
Using trapezoidal collocation, we approximate the change in state in the continuous function $\dot x = v$ as $x_{k+1}-x_k \approx \frac{1}{2} h_k (v_{k+1}+v_k)$. Also, for $\ddot x = u$, we have $v_{k+1}-v_k \approx \frac{1}{2} h_k (u_{k+1}+u_k)$. The boundary conditions are $x_0 = v_0 = v_N = 0$ and $x_N = 1$. Discretizing the objective function, we have:
<br> $$\min_{u(t)} \int_{t_0}^{t_N} u^2 (\tau) d\tau \approx \min_{u_0 .. u_N} \sum_{k=0}^{N-1} \frac{1}{2} h_k (u_k^2+u_{k+1}^2)$$

4. *Initialization* - to kickstart the nonlinear program, we need initial guesses. A simple trajectory for our problem is a ramp. Evaluating the guessed trajectory at each collocation point, we can obtain values to pass to the nonlinear program.


In [1]:
import pybullet as p
import pybullet_data
physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #used by loadURDF
p.setGravity(0, 0, -9.81)

# Import cart pole
planeId = p.loadURDF("plane.urdf")
startPos = [0,0,1.5]
startOrientation = p.getQuaternionFromEuler([0,0,0])
cartPoleId = p.loadURDF("./resource/cart_pole.urdf",startPos, startOrientation)

p.setJointMotorControl2(cartPoleId, 0, p.VELOCITY_CONTROL, force=0)
p.setJointMotorControl2(cartPoleId, 1, p.VELOCITY_CONTROL, force=0)

p.setRealTimeSimulation(1)
for i in range(10000000):
    pos, orn = p.getBasePositionAndOrientation(cartPoleId)
#     print(cubePos,cubeOrn)
p.disconnect()